In [50]:
import pandas as pd
import numpy as np

In [2]:
target_date_str = '20150302'
target_datetime = pd.to_datetime(target_date_str, format='%Y%m%d')

In [3]:
orders_df = pd.read_csv(f'../moex_dataset/OrderLog{target_date_str}.txt', index_col='NO')

In [4]:
trades_df = pd.read_csv(f'../moex_dataset/TradeLog{target_date_str}.txt')

## 1. Preprocessing

### 1.1 Add DATETIME column with datetime

In [5]:
orders_df['DATETIME'] = pd.to_datetime(target_date_str + orders_df['TIME'].astype(str).str.zfill(9), format='%Y%m%d%H%M%S%f')

In [6]:
orders_df[(orders_df.SECCODE == 'SBER') & (orders_df.TIME > 100000000) & (orders_df.TIME < 100002000)].DATETIME

NO
10453   2015-03-02 10:00:00.071
10455   2015-03-02 10:00:00.073
10457   2015-03-02 10:00:00.095
10460   2015-03-02 10:00:00.110
10466   2015-03-02 10:00:00.139
                  ...          
13079   2015-03-02 10:00:01.904
13103   2015-03-02 10:00:01.931
13104   2015-03-02 10:00:01.931
13105   2015-03-02 10:00:01.931
13130   2015-03-02 10:00:01.960
Name: DATETIME, Length: 243, dtype: datetime64[ns]

In [7]:
trades_df['DATETIME'] = pd.to_datetime(target_date_str + trades_df['TIME'].astype(str).str.zfill(6), format='%Y%m%d%H%M%S')

In [8]:
trades_df.DATETIME

0        2015-03-02 10:00:00
1        2015-03-02 10:00:00
2        2015-03-02 10:00:00
3        2015-03-02 10:00:00
4        2015-03-02 10:00:00
                 ...        
409249   2015-03-02 18:39:58
409250   2015-03-02 18:39:58
409251   2015-03-02 18:39:58
409252   2015-03-02 18:39:58
409253   2015-03-02 18:39:58
Name: DATETIME, Length: 409254, dtype: datetime64[ns]

In [21]:
trades_df

,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME,DATETIME
0,2460134732,URKA,100000,3576,10467,171.1000,320,2015-03-02 10:00:00
1,2460134733,SBERP,100000,6756,10471,53.8200,700,2015-03-02 10:00:00
2,2460134734,RSTI,100000,9605,10473,0.6445,472000,2015-03-02 10:00:00
3,2460134735,TATN,100000,2088,10487,318.8500,1940,2015-03-02 10:00:00
4,2460134736,MOEX,100000,1135,10488,76.2100,330,2015-03-02 10:00:00
...,...,...,...,...,...,...,...,...
409249,2460556953,LSRG,183958,6817292,6920224,582.0000,104,2015-03-02 18:39:58
409250,2460556954,SNGSP,183958,6920241,6920115,41.3450,100,2015-03-02 18:39:58
409251,2460556955,MTLR,183958,6895955,6920251,83.0100,606,2015-03-02 18:39:58
409252,2460556956,TRNFP,183958,6920271,6919704,137490.0000,1,2015-03-02 18:39:58


In [9]:
orders_df[(orders_df.SECCODE == 'SBER') & (orders_df.ACTION == 2)]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
10603,SBER,B,100000550,10567,2,0.00,100,2.460135e+09,76.34,2015-03-02 10:00:00.550
10604,SBER,S,100000550,10465,2,76.34,100,2.460135e+09,76.34,2015-03-02 10:00:00.550
10611,SBER,B,100000566,10492,2,76.33,10000,2.460135e+09,76.33,2015-03-02 10:00:00.566
10612,SBER,S,100000566,10572,2,76.33,10000,2.460135e+09,76.33,2015-03-02 10:00:00.566
10645,SBER,B,100000615,10492,2,76.33,50,2.460135e+09,76.33,2015-03-02 10:00:00.615
...,...,...,...,...,...,...,...,...,...,...
14200411,SBER,S,183958468,6920196,2,0.00,190,2.460557e+09,76.15,2015-03-02 18:39:58.468
14200412,SBER,B,183958468,6919675,2,76.12,10,2.460557e+09,76.12,2015-03-02 18:39:58.468
14200413,SBER,S,183958468,6920196,2,0.00,10,2.460557e+09,76.12,2015-03-02 18:39:58.468


In [10]:
sber_orders_df = orders_df[(orders_df.SECCODE == 'SBER')]

In [76]:
grouped_orders = sber_orders_df[sber_orders_df.ACTION == 2][['ORDERNO', 'VOLUME']].groupby('ORDERNO').sum('VOLUME')
merged_df = sber_orders_df.merge(grouped_orders, left_on='ORDERNO', right_on='ORDERNO', how='left')
icebergs = merged_df[(merged_df.VOLUME_y > merged_df.VOLUME_x) & (merged_df.ACTION == 1)][['ORDERNO']]
icebergs.loc[:, 'IS_ICEBERG'] = np.True_
merged_df = merged_df.merge(icebergs, left_on='ORDERNO', right_on='ORDERNO', how='left')
merged_df.rename(columns={'VOLUME_x': 'VOLUME', 'VOLUME_y': 'VOLUME_SUM'}, inplace=True)


In [77]:
merged_df[merged_df.IS_ICEBERG.notna()]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME,VOLUME_SUM,IS_ICEBERG
879,SBER,B,100000733,10668,1,76.40,6440,NaN,NaN,2015-03-02 10:00:00.733,6980.0,True
880,SBER,B,100000733,10668,2,76.40,5980,2.460135e+09,76.35,2015-03-02 10:00:00.733,6980.0,True
882,SBER,B,100000733,10668,2,76.40,1000,2.460135e+09,76.36,2015-03-02 10:00:00.733,6980.0,True
884,SBER,B,100000733,10669,1,76.40,6440,NaN,NaN,2015-03-02 10:00:00.733,12460.0,True
885,SBER,B,100000733,10669,2,76.40,50,2.460135e+09,76.37,2015-03-02 10:00:00.733,12460.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1013404,SBER,B,183840610,6884671,2,76.30,12220,2.460553e+09,76.29,2015-03-02 18:38:40.610,12220.0,True
1013588,SBER,B,183849064,6869347,2,76.21,1110000,2.460553e+09,76.21,2015-03-02 18:38:49.064,1138440.0,True
1014195,SBER,B,183853499,6890890,1,76.20,10000,NaN,NaN,2015-03-02 18:38:53.499,419410.0,True
1014196,SBER,B,183853499,6890890,2,76.20,90,2.460554e+09,76.19,2015-03-02 18:38:53.499,419410.0,True


In [12]:
merged_df[(merged_df.ACTION == 1) & (~merged_df.VOLUME_y.isna()) & (merged_df.VOLUME_y > merged_df.VOLUME_x) & (merged_df.TIME > 101000000)]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME_x,TRADENO,TRADEPRICE,DATETIME,VOLUME_y
NO,,,,,,,,,,,
1360591,SBER,B,101018963,678751,1,76.28,1290,NaN,NaN,2015-03-02 10:10:18.963,1640.0
1420373,SBER,B,101057683,708345,1,76.28,1290,NaN,NaN,2015-03-02 10:10:57.683,5220.0
1797579,SBER,S,101416786,895519,1,76.49,23810,NaN,NaN,2015-03-02 10:14:16.786,54060.0
1835060,SBER,S,101437273,914358,1,76.49,23810,NaN,NaN,2015-03-02 10:14:37.273,41180.0
1859884,SBER,S,101452119,926731,1,76.48,5000,NaN,NaN,2015-03-02 10:14:52.119,21600.0
...,...,...,...,...,...,...,...,...,...,...,...
14092422,SBER,B,183806623,6869346,1,76.21,30000,NaN,NaN,2015-03-02 18:38:06.623,31660.0
14092437,SBER,B,183806623,6869347,1,76.21,30000,NaN,NaN,2015-03-02 18:38:06.623,1138440.0
14124212,SBER,B,183840610,6884670,1,76.30,5500,NaN,NaN,2015-03-02 18:38:40.610,14050.0


In [20]:
merged_df[merged_df.ORDERNO == 6888856][-50:]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME_x,TRADENO,TRADEPRICE,DATETIME,VOLUME_y
NO,,,,,,,,,,,
14134531,SBER,S,183850267,6888856,2,76.2,800,2.460554e+09,76.2,2015-03-02 18:38:50.267,1867280.0
14134551,SBER,S,183850282,6888856,2,76.2,500,2.460554e+09,76.2,2015-03-02 18:38:50.282,1867280.0
14134590,SBER,S,183850314,6888856,2,76.2,50,2.460554e+09,76.2,2015-03-02 18:38:50.314,1867280.0
14135287,SBER,S,183851173,6888856,2,76.2,400,2.460554e+09,76.2,2015-03-02 18:38:51.173,1867280.0
14135472,SBER,S,183851376,6888856,2,76.2,20,2.460554e+09,76.2,2015-03-02 18:38:51.376,1867280.0
14136010,SBER,S,183852110,6888856,2,76.2,39900,2.460554e+09,76.2,2015-03-02 18:38:52.110,1867280.0
14136066,SBER,S,183852170,6888856,2,76.2,50,2.460554e+09,76.2,2015-03-02 18:38:52.170,1867280.0
14136220,SBER,S,183852310,6888856,2,76.2,10,2.460554e+09,76.2,2015-03-02 18:38:52.310,1867280.0
14136508,SBER,S,183852680,6888856,2,76.2,2100,2.460554e+09,76.2,2015-03-02 18:38:52.680,1867280.0


In [14]:
merged_df[merged_df.ORDERNO == 6869347]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME_x,TRADENO,TRADEPRICE,DATETIME,VOLUME_y
NO,,,,,,,,,,,
14092437,SBER,B,183806623,6869347,1,76.21,30000,NaN,NaN,2015-03-02 18:38:06.623,1138440.0
14092438,SBER,B,183806623,6869347,2,76.21,7120,2.460552e+09,76.20,2015-03-02 18:38:06.623,1138440.0
14092440,SBER,B,183806623,6869347,2,76.21,1840,2.460552e+09,76.20,2015-03-02 18:38:06.623,1138440.0
14092442,SBER,B,183806623,6869347,2,76.21,1300,2.460552e+09,76.21,2015-03-02 18:38:06.623,1138440.0
14092446,SBER,B,183806625,6869347,2,76.21,3000,2.460552e+09,76.21,2015-03-02 18:38:06.625,1138440.0
14092449,SBER,B,183806625,6869347,2,76.21,12040,2.460552e+09,76.21,2015-03-02 18:38:06.625,1138440.0
14092476,SBER,B,183806643,6869347,2,76.21,100,2.460552e+09,76.21,2015-03-02 18:38:06.643,1138440.0
14093272,SBER,B,183807406,6869347,2,76.21,1200,2.460552e+09,76.21,2015-03-02 18:38:07.406,1138440.0
14093541,SBER,B,183807863,6869347,2,76.21,400,2.460552e+09,76.21,2015-03-02 18:38:07.863,1138440.0


In [15]:
chunk = merged_df[(merged_df.ORDERNO == 6869347) & (merged_df.TRADENO.notna())]
chunk.TRADENO = chunk.TRADENO.astype(int)
chunk

/tmp/ipykernel_1896/3746825173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.TRADENO = chunk.TRADENO.astype(int)


,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME_x,TRADENO,TRADEPRICE,DATETIME,VOLUME_y
NO,,,,,,,,,,,
14092438,SBER,B,183806623,6869347,2,76.21,7120,2460552288,76.20,2015-03-02 18:38:06.623,1138440.0
14092440,SBER,B,183806623,6869347,2,76.21,1840,2460552289,76.20,2015-03-02 18:38:06.623,1138440.0
14092442,SBER,B,183806623,6869347,2,76.21,1300,2460552290,76.21,2015-03-02 18:38:06.623,1138440.0
14092446,SBER,B,183806625,6869347,2,76.21,3000,2460552291,76.21,2015-03-02 18:38:06.625,1138440.0
14092449,SBER,B,183806625,6869347,2,76.21,12040,2460552292,76.21,2015-03-02 18:38:06.625,1138440.0
14092476,SBER,B,183806643,6869347,2,76.21,100,2460552293,76.21,2015-03-02 18:38:06.643,1138440.0
14093272,SBER,B,183807406,6869347,2,76.21,1200,2460552357,76.21,2015-03-02 18:38:07.406,1138440.0
14093541,SBER,B,183807863,6869347,2,76.21,400,2460552359,76.21,2015-03-02 18:38:07.863,1138440.0
14094889,SBER,B,183809985,6869347,2,76.21,100,2460552412,76.21,2015-03-02 18:38:09.985,1138440.0


In [16]:
orders_df[orders_df.TRADENO == 2460553489]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
14132885,SBER,B,183849064,6869347,2,76.21,1110000,2.460553e+09,76.21,2015-03-02 18:38:49.064
14132886,SBER,S,183849064,6888856,2,76.20,1110000,2.460553e+09,76.21,2015-03-02 18:38:49.064


In [17]:
merged_df[(merged_df.ORDERNO == 10669) & (merged_df.TRADENO.notna())].TRADENO.astype(int)

NO
10755    2460134765
10757    2460134766
10759    2460134767
10761    2460134768
10763    2460134769
10765    2460134770
10767    2460134771
Name: TRADENO, dtype: int64

In [38]:
merged_df.loc[merged_df.ORDERNO == 9624]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME_x,TRADENO,TRADEPRICE,DATETIME,VOLUME_y
NO,,,,,,,,,,,
9624,SBER,S,100000000,9624,1,76.39,10000,NaN,NaN,2015-03-02 10:00:00.000,10000.0
10768,SBER,S,100000733,9624,2,76.39,10000,2.460135e+09,76.39,2015-03-02 10:00:00.733,10000.0


In [79]:
from collections import defaultdict
from dataclasses import dataclass


@dataclass
class Order:
    price: float
    volume: int


class OrderBook:
    def __init__(self, price_max: float, price_min: float, price_delta: float):
        self._buy_book = defaultdict(dict)
        self._sell_book = defaultdict(dict)
        self._top_buy = None
        self._bottom_sell = None
        self._price_delta = price_delta
        self._price_max = price_max
        self._price_min = price_min

    def _find_bottom_sell(self) -> float:
        bottom_sell = self._bottom_sell
        while self._sell_book.get(bottom_sell) is None and bottom_sell <= self._price_max:
            bottom_sell += self._price_delta
        return bottom_sell

    def _find_top_buy(self) -> float:
        top_buy = self._top_buy
        while self._buy_book.get(top_buy) is None and top_buy >= self._price_min:
            top_buy -= self._price_delta
        return top_buy
    
    def get_best_bid(self) -> float | None:
        return self._top_buy
    
    def get_best_ask(self):
        return self._bottom_sell
    
    def add_buy_order(self, price: float, orderno: int, volume: int):
        if self._top_buy is None or price > self._top_buy:
            self._top_buy = price
        self._buy_book[price][orderno] = Order(price, volume)
    
    def add_sell_order(self, price: float, orderno: int, volume: int):
        if self._bottom_sell is None or price < self._bottom_sell:
            self._bottom_sell = price
        self._sell_book[price][orderno] = Order(price, volume)
    
    def cancel_buy_order(self, price: float, orderno: int):
        self._buy_book[price].pop(orderno)
        if len(self._buy_book[price]) == 0:
            self._buy_book.pop(price)
        self._top_buy = self._find_top_buy()
    
    def cancel_sell_order(self, price: float, orderno: int):
        self._sell_book[price].pop(orderno)
        if len(self._sell_book[price]) == 0:
            self._sell_book.pop(price)
        self._bottom_sell = self._find_bottom_sell()

    def trade_buy(
            self,
            price: float,
            orderno: int,
            volume: int,
            is_iceberg: bool = False,
    ):
        self._buy_book[price][orderno].volume -= volume
        if not is_iceberg:
            assert self._buy_book[price][orderno].volume >= 0
        if self._buy_book[price][orderno].volume == 0:
            self._buy_book[price].pop(orderno)
            if len(self._buy_book[price]) == 0:
                self._buy_book.pop(price)
        self._bottom_sell = self._find_bottom_sell()
    
    def trade_sell(
            self,
            price: float,
            orderno: int,
            volume: int,
            is_iceberg: bool = False,
    ):
        self._sell_book[price][orderno].volume -= volume
        if not is_iceberg:
            assert self._sell_book[price][orderno].volume >= 0
        if self._sell_book[price][orderno].volume == 0:
            self._sell_book[price].pop(orderno)
            if len(self._sell_book[price]) == 0:
                self._sell_book.pop(price)
        self._top_buy = self._find_top_buy()


In [49]:
merged_df[(merged_df.TIME >= 100000733) & (merged_df.TIME < 100001000)].to_csv('iceberg.csv')

In [42]:
orders_df[orders_df.TRADENO == 2460134764]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
10752,SBER,B,100000733,10668,2,76.40,1000,2.460135e+09,76.36,2015-03-02 10:00:00.733
10753,SBER,S,100000733,10452,2,76.36,1000,2.460135e+09,76.36,2015-03-02 10:00:00.733


In [43]:
orders_df[orders_df.ORDERNO == 10668]

,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATETIME
NO,,,,,,,,,,
10749,SBER,B,100000733,10668,1,76.4,6440,NaN,NaN,2015-03-02 10:00:00.733
10750,SBER,B,100000733,10668,2,76.4,5980,2.460135e+09,76.35,2015-03-02 10:00:00.733
10752,SBER,B,100000733,10668,2,76.4,1000,2.460135e+09,76.36,2015-03-02 10:00:00.733


### ACTION
0=Cancel order  
1=Create order  
2=Trade  

In [93]:
import tqdm


best_bid = []
best_ask = []
cur_best_bid = None
cur_best_ask = None

price_max = 100
price_min = 0
price_delta = 0.01
order_book = OrderBook(price_max, price_min, price_delta)
counter = 0
time_intervals = []
is_anomaly = None
for idx, row in tqdm.tqdm(merged_df[(merged_df.SECCODE == 'SBER')].iterrows()):
    buysell = row['BUYSELL']
    orderno = row['ORDERNO']
    price = row['PRICE']
    volume = row['VOLUME']
    action = row['ACTION']
    is_iceberg = row['IS_ICEBERG']
    time = row['TIME']
    best_bid = order_book.get_best_bid()
    best_ask = order_book.get_best_ask()
    if row['ACTION'] == 0:
        if buysell == 'B':
            order_book.cancel_buy_order(price, orderno)
        elif buysell == 'S':
            order_book.cancel_sell_order(price, orderno)
        else:
            raise ValueError(f'Unknown action: {row["ACTION"]}')
        best_bid = order_book.get_best_bid()
        best_ask = order_book.get_best_ask()
    elif row['ACTION'] == 1:
        if buysell == 'B':
            order_book.add_buy_order(price, orderno, volume)
        elif buysell == 'S':
            order_book.add_sell_order(price, orderno, volume)
        else:
            raise ValueError(f'Unknown action: {row["ACTION"]}')
        best_bid = order_book.get_best_bid()
        best_ask = order_book.get_best_ask()
    elif row['ACTION'] == 2:
        if buysell == 'B':
            order_book.trade_buy(price, orderno, volume, is_iceberg)
        elif buysell == 'S':
            order_book.trade_sell(price, orderno, volume, is_iceberg)
        else:
            raise ValueError(f'Unknown action: {row["ACTION"]}')
    else:
        raise ValueError(f'Unknown action: {row["ACTION"]}')
    counter += 1
    if best_bid is not None and best_ask is not None and best_bid > best_ask:
        anomaly_counter += 1
    print(best_bid, best_ask, time, action, price, buysell)
    if counter >= 10000:
        break


1970it [00:01, 2911.00it/s]

70.22 None 100000000 1 70.22 B
70.22 78.28 100000000 1 78.28 S
73.0 78.28 100000000 1 73.0 B
73.0 76.49 100000000 1 76.49 S
73.0 76.49 100000000 1 77.33 S
74.39 76.49 100000000 1 74.39 B
74.39 76.49 100000000 1 77.4 S
74.39 76.49 100000000 1 77.2 S
74.39 76.49 100000000 1 77.0 S
75.26 76.49 100000000 1 75.26 B
75.26 76.49 100000000 1 74.39 B
75.26 76.49 100000000 1 76.8 S
75.26 76.49 100000000 1 74.39 B
75.26 76.49 100000000 1 74.39 B
75.26 76.49 100000000 1 72.31 B
75.26 76.49 100000000 1 69.98 B
75.26 76.49 100000000 1 74.39 B
75.4 76.49 100000000 1 75.4 B
75.4 76.49 100000000 1 75.2 B
75.4 76.49 100000000 1 75.0 B
75.4 76.49 100000000 1 74.8 B
75.4 76.49 100000000 1 74.6 B
75.4 76.49 100000000 1 74.4 B
75.4 76.49 100000000 1 74.2 B
75.4 76.49 100000000 1 74.0 B
75.4 76.49 100000000 1 76.74 S
75.4 76.49 100000000 1 75.05 B
75.4 76.49 100000000 1 76.86 S
75.4 76.49 100000000 1 74.5 B
75.4 76.49 100000000 1 74.39 B
75.4 76.49 100000000 1 74.1 B
75.4 76.49 100000000 1 77.49 S
75.4 76.49

3804it [00:01, 5367.18it/s]

76.44 100005977 1 76.32 B
76.32 76.44 100005990 0 75.63 B
76.32 76.44 100005991 1 76.48 S
76.32 76.27 100005993 1 76.27 S
76.32 76.27 100005993 2 76.32 B
76.32 76.27 100005993 2 76.27 S
-2.1388203708117004e-12 76.27 100005993 2 76.3 B
-2.1388203708117004e-12 76.27 100005993 2 76.27 S
-2.1388203708117004e-12 100.00000000001214 100005997 0 76.44 S
76.31 100.00000000001214 100005999 1 76.31 B
76.31 76.44 100005999 1 76.44 S
76.31 76.29 100006012 1 76.29 S
76.31 76.29 100006012 2 76.31 B
76.31 76.29 100006012 2 76.29 S
76.31 76.29 100006014 1 76.29 S
76.31 76.29 100006014 2 76.31 B
76.31 76.29 100006014 2 76.29 S
76.31 100.00000000001214 100006014 0 76.44 S
76.3 100.00000000001214 100006015 0 76.31 B
76.3 76.28 100006016 1 76.28 S
76.3 76.28 100006016 2 76.3 B
76.3 76.28 100006016 2 76.28 S
76.3 76.28 100006021 1 76.44 S
76.3 76.28 100006021 0 76.18 B
76.31 76.28 100006021 1 76.31 B
76.31 76.28 100006030 1 76.16 B
76.31 76.28 100006051 1 76.47 S
76.31 76.28 100006060 1 76.29 S
76.31 76.28 

5818it [00:01, 7399.26it/s]

76.32 76.35 100025011 1 76.29 B
76.32 76.35 100025028 0 76.29 B
76.32 76.35 100025029 1 76.29 B
76.32 76.35 100025030 0 75.44 B
76.32 76.35 100025031 1 75.47 B
76.33 76.35 100025046 1 76.33 B
76.33 76.35 100025054 0 75.48 B
76.33 76.35 100025056 1 75.5 B
76.33 76.35 100025057 0 75.49 B
76.33 76.35 100025058 1 75.51 B
76.33 76.35 100025069 0 76.29 B
76.33 76.35 100025070 1 76.29 B
76.33 76.35 100025083 0 76.06 B
76.33 76.35 100025105 0 75.5 B
76.33 76.35 100025106 1 75.52 B
76.33 76.35 100025107 0 75.51 B
76.33 76.35 100025108 1 75.53 B
76.33 76.35 100025113 0 76.29 B
76.33 76.35 100025114 1 76.29 B
76.33 76.35 100025119 0 76.29 B
76.33 76.35 100025120 1 76.07 B
76.33 76.35 100025121 1 76.29 B
76.33 76.35 100025126 0 76.29 B
76.33 76.35 100025127 1 76.29 B
76.33 76.35 100025133 0 76.29 B
76.33 76.35 100025133 0 76.41 S
76.33 76.35 100025135 1 76.29 B
76.33 76.35 100025135 1 76.41 S
76.33 76.35 100025138 0 76.41 S
76.33 76.35 100025139 1 76.4 S
76.33 76.35 100025142 0 76.29 B
76.33 76.35

7841it [00:01, 8721.62it/s]

76.23 76.31 100039326 1 76.32 S
76.23 76.29 100039327 1 76.29 S
76.23 76.29 100039327 0 76.04 B
76.23 76.29 100039328 1 76.04 B
76.23 76.29 100039328 0 76.19 B
76.23 76.29 100039330 1 76.29 S
76.23 76.29 100039330 1 76.29 S
76.23 76.29 100039332 0 76.32 S
76.23 76.29 100039333 1 76.74 S
76.23 76.29 100039334 0 76.31 S
76.23 76.29 100039335 1 76.19 B
76.23 76.29 100039336 1 76.31 S
76.23 76.29 100039346 1 76.38 S
76.23 76.29 100039353 0 76.19 B
76.23 76.29 100039354 0 76.74 S
76.23 76.29 100039355 1 76.73 S
76.23 76.29 100039359 0 76.29 S
76.23 76.29 100039368 0 76.13 B
76.23 76.29 100039378 0 76.74 S
76.23 76.29 100039379 1 76.72 S
76.23 76.29 100039380 0 76.73 S
76.23 76.29 100039381 1 76.71 S
76.23 76.29 100039391 0 76.29 S
76.23 76.29 100039393 1 76.08 B
76.23 76.29 100039394 0 76.04 B
76.23 76.29 100039395 1 76.12 B
76.23 76.22 100039397 1 76.22 S
76.23 76.22 100039397 2 76.26 B
76.23 76.22 100039397 2 76.22 S
76.23 76.22 100039477 1 76.26 S
76.23 76.22 100039477 2 76.26 B
76.23 10

9999it [00:01, 5407.83it/s]

76.16 76.19 100101484 0 76.21 S
76.16 76.18 100101484 1 76.18 S
76.16 76.18 100101484 1 76.12 B
76.16 76.18 100101487 0 76.12 B
76.16 76.18 100101488 1 76.03 B
76.16 76.18 100101493 0 76.19 S
76.16 76.18 100101493 1 76.21 S
76.16 100.0000000000122 100101493 0 76.18 S
76.16 76.2 100101494 1 76.2 S
76.16 76.2 100101494 0 76.21 S
76.16 76.19 100101494 1 76.19 S
76.16 76.19 100101494 0 76.03 B
76.16 76.19 100101500 0 76.2 S
76.16 76.19 100101500 1 76.21 S
76.16 100.00000000001218 100101500 0 76.19 S
76.16 76.19 100101500 1 76.19 S
76.16 76.19 100101501 1 76.12 B
76.16 76.19 100101501 0 76.21 S
76.16 76.18 100101501 1 76.18 S
76.16 76.18 100101501 0 76.19 S
76.16 100.0000000000122 100101502 0 76.18 S
76.16 76.21 100101502 1 76.21 S
76.16 76.21 100101502 0 76.12 B
76.16 76.2 100101502 1 76.2 S
76.16 76.2 100101503 0 76.21 S
76.16 76.19 100101503 1 76.19 S
76.16 76.19 100101503 0 76.2 S
76.16 76.19 100101503 1 76.12 B
76.16 100.00000000001218 100101504 0 76.19 S
76.16 76.21 100101504 1 76.21 

In [85]:
order_book.get_best_ask()

100.00000000001425

In [72]:
trades_df[trades_df.SECCODE == 'SBER']

,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME,DATETIME
11,2460134743,SBER,100000,10567,10465,76.34,100,2015-03-02 10:00:00
12,2460134744,SBER,100000,10492,10572,76.33,10000,2015-03-02 10:00:00
13,2460134745,SBER,100000,10492,10601,76.33,50,2015-03-02 10:00:00
16,2460134749,SBER,100000,10647,10465,76.34,900,2015-03-02 10:00:00
17,2460134750,SBER,100000,10647,9868,76.35,7100,2015-03-02 10:00:00
...,...,...,...,...,...,...,...,...
409242,2460556946,SBER,183958,6920165,6920122,76.18,10,2015-03-02 18:39:58
409244,2460556948,SBER,183958,6920120,6920192,76.15,10,2015-03-02 18:39:58
409245,2460556949,SBER,183958,6920120,6920196,76.15,190,2015-03-02 18:39:58
409246,2460556950,SBER,183958,6919675,6920196,76.12,10,2015-03-02 18:39:58


In [59]:
trades_df[trades_df.SECCODE == 'SBER'].TIME.max()

np.int64(183958)

In [25]:
orders_df[orders_df['SECCODE'] == 'SBER'].shape

(1994350, 9)

In [14]:
trades_df[trades_df['SECCODE'] == 'SBER'].shape

(82364, 7)

In [49]:
orders_df.TIME.max()

np.int64(184459956)

In [33]:
orders_df[orders_df.SECCODE == 'SBER'].TRADENO.notna().sum()

np.int64(204538)

In [ ]:
full_channel_df.time = pd.to_datetime(full_channel_df.time)
full_channel_df['timestamp'] = full_channel_df['time'].astype('int64')

ticker_df.time = pd.to_datetime(ticker_df.time)
ticker_df['timestamp'] = ticker_df['time'].astype('int64')